In [41]:
## Libraries import

import carla, time, pygame, cv2, math, random, os, traceback, logging
import paho.mqtt.client as mqtt
import numpy as np
import tkinter as tk
from ultralytics.models.yolo import YOLO

logging.getLogger("ultralytics").setLevel(logging.WARNING)

In [42]:
BROKER = "test.mosquitto.org"
PORT = 1883
TOPIC = "pedestrian_monitoring/"

VIEW_WIDTH = tk.Tk().winfo_screenwidth()# //2 # //6 on wide screen //3 on normal screen
VIEW_HEIGHT = tk.Tk().winfo_screenheight()# //2
VIEW_FOV = 90

In [43]:
## Carla set up
mqtt_client = mqtt.Client()
def on_message(client, userdata, message):
    print(f"Received: {message.payload.decode()} on {message.topic}")

mqtt_client.on_message = on_message

try:
    mqtt_client.connect(BROKER, PORT, 60)
    mqtt_client.loop_start()
    print(f"Listening to {TOPIC} on {BROKER}...")
except Exception as e:
    print("Connection failed:", e)

try:
    client = carla.Client('localhost', 2000)
    client.set_timeout(10.0)
    world = client.get_world()
except Exception as e:
    print(f"Failed to connect to CARLA: {e}")
    exit(1)

night_scenario = carla.WeatherParameters(
    sun_azimuth_angle=180.0,
    sun_altitude_angle=-90.0,
)

sunny_weather = carla.WeatherParameters(
    cloudiness=0.0,
    precipitation=0.0,
    precipitation_deposits=0.0,
    wind_intensity=5.0,
    sun_altitude_angle=70.0
)

rainy_weather = carla.WeatherParameters(
    cloudiness=80.0,
    precipitation=100.0,
    precipitation_deposits=80.0,
    wind_intensity=30.0,
    sun_altitude_angle=20.0
)

foggy_weather = carla.WeatherParameters(
    cloudiness=70.0,
    precipitation=0.0,
    precipitation_deposits=0.0,
    wind_intensity=10.0,
    fog_density=80.0,
    fog_distance=10.0,
    fog_falloff=2.0,
    sun_altitude_angle=15.0
)

world.set_weather(sunny_weather)

if not all(os.path.exists(f) for f in ["yolov8n.pt"]):
    print("YOLO model files not found!")
    exit(1)

world = client.get_world()
spectator = world.get_spectator()
capture = True
stored_image = None
stored_rgb_image = None
stored_depth_image = None
backwards_only_flag = False
braking_flag = False

c:\Users\valentina.pieri5\AppData\Local\anaconda3\envs\carla-env\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  


Listening to pedestrian_monitoring/ on test.mosquitto.org...


In [44]:
### image processing

def render(image):
    if image is not None:
        array = np.frombuffer(image.raw_data, dtype=np.uint8)
        array = np.reshape(array, (image.height, image.width, 4))
        array = array[:, :, :3]
        array = array[:, :, ::-1]

        return array

def image_processing(image, target_size):
    ih, iw = target_size
    h, w, _ = image.shape

    scale = min(iw / w, ih / h)
    nw, nh = int(scale * w), int(scale * h)

    image_resized = cv2.resize(image, (nw, nh), interpolation = cv2.INTER_LINEAR)

    image_padded = np.full((ih, iw, 3), 128.0)
    dw, dh = (iw - nw) // 2, (ih - nh) // 2
    image_padded[dh:nh+dh, dw:nw+dw, :] = image_resized

    image_padded = image_padded / 255.0
    return image_padded.astype(np.float32)

def set_image(img):
    global stored_image
    global capture
    if capture:
        stored_image = img
        capture = False

def set_rgb_image(image):
    """Stores the latest RGB image."""
    global stored_rgb_image
    stored_rgb_image = image

def set_depth_image(image):
    """Stores the latest Depth image."""
    global stored_depth_image
    stored_depth_image = image

def get_depth_at_pixel(x, y):
    """
    Retrieves the depth value (in meters) for a given pixel (x, y).
    Assumes the stored_depth_image is in BGRA format.
    """
    if stored_depth_image is None:
        return None

    if x < 0 or x >= stored_depth_image.width or y < 0 or y >= stored_depth_image.height:
        raise ValueError("Pixel coordinates are out of bounds.")

    depth_array = np.frombuffer(stored_depth_image.raw_data, dtype=np.uint8)
    depth_array = np.reshape(depth_array, (stored_depth_image.height, stored_depth_image.width, 4))

    blue  = depth_array[y, x, 0]
    green = depth_array[y, x, 1]
    red   = depth_array[y, x, 2]

    normalized_depth = (red + green * 256 + blue * 256**2) / (256**3 - 1)

    depth_in_meters = normalized_depth * 1000.0

    return depth_in_meters

In [45]:
### setup simulation env

def setup_car():
    """
    Spawns actor-vehicle to be controled.
    """
    car_bp = world.get_blueprint_library().filter('vehicle.*')[0]
    location = random.choice(world.get_map().get_spawn_points())
    car = world.spawn_actor(car_bp, location)
    return car

def spawn_walker(world):
    blueprint_library = world.get_blueprint_library()

    walker_blueprints = list(blueprint_library.filter('walker.pedestrian.*'))

    if not walker_blueprints:
        print("No pedestrian blueprints available.")
        return None, None

    walker_bp = random.choice(walker_blueprints)

    if walker_bp.has_attribute("speed"):
        speed = random.uniform(0.8, 2.0)
        walker_bp.set_attribute('speed', str(speed))
    else:
        speed = 1.0

    spawn_points = world.get_map().get_spawn_points()
    if not spawn_points:
        print("Warning: No spawn points available for pedestrians. Cannot spawn walker.")
        return None, None

    spawn_point = random.choice(spawn_points)

    walker = world.try_spawn_actor(walker_bp, spawn_point)
    if walker is None:
        print("Failed to spawn walker at the chosen spawn point.")
        return None, None

    controller_bp = blueprint_library.find('controller.ai.walker')
    if controller_bp is None:
        print("No walker controller blueprint found.")
        return walker, None

    controller = world.try_spawn_actor(controller_bp, carla.Transform(), walker)
    if controller:
        controller.start()
        destination = world.get_random_location_from_navigation()
        if destination:
            controller.go_to_location(destination)
        controller.set_max_speed(speed)

    return walker, controller

def setup_camera(car):
    """
    Spawns both an RGB and a Depth camera on the vehicle.
    """
    camera_transform = carla.Transform(carla.Location(x=1, y=-0.4, z=1.2), carla.Rotation())

    blueprint_library = world.get_blueprint_library()

    rgb_bp = blueprint_library.find('sensor.camera.rgb')
    rgb_bp.set_attribute('image_size_x', str(VIEW_WIDTH))
    rgb_bp.set_attribute('image_size_y', str(VIEW_HEIGHT))
    rgb_camera = world.spawn_actor(rgb_bp, camera_transform, attach_to=car)
    rgb_camera.listen(lambda image: set_rgb_image(image))

    depth_bp = blueprint_library.find('sensor.camera.depth')
    depth_bp.set_attribute('image_size_x', str(VIEW_WIDTH))
    depth_bp.set_attribute('image_size_y', str(VIEW_HEIGHT))
    depth_camera = world.spawn_actor(depth_bp, camera_transform, attach_to=car)
    depth_camera.listen(lambda image: set_depth_image(image))

    calibration = np.identity(3)
    calibration[0, 2] = VIEW_WIDTH / 2.0
    calibration[1, 2] = VIEW_HEIGHT / 2.0
    calibration[0, 0] = calibration[1, 1] = VIEW_WIDTH / (2.0 * np.tan(VIEW_FOV * np.pi / 360.0))
    rgb_camera.calibration = calibration
    depth_camera.calibration = calibration

    return rgb_camera, depth_camera

def move_spectator_to(transform, spectator, distance=5.0, x=0, y=0, z=4, yaw=0, pitch=-30, roll=0):
    back_location = transform.location - transform.get_forward_vector() * distance

    back_location.x += x
    back_location.y += y
    back_location.z += z
    transform.rotation.yaw += yaw
    transform.rotation.pitch = pitch
    transform.rotation.roll = roll

    spectator_transform = carla.Transform(back_location, transform.rotation)

    spectator.set_transform(spectator_transform)

In [46]:
### car controllers

def control_car(car, backwards_only=False):
    """
    Applies control to main car based on pygame pressed keys.
    Will return True If ESCAPE is hit, otherwise False to end main loop.
    """
    control = car.get_control()

    control.brake = 0.0
    control.steer = 0.0

    move_spectator_to(car.get_transform(), spectator)
    for event in pygame.event.get():
        if event.type == pygame.KEYDOWN:
            key = event.key
            if backwards_only:
                if key == pygame.K_s:
                    control.reverse = True
                    control.throttle = 0.6
                if key == pygame.K_a:
                    control.throttle = 0.4
                    control.steer = -0.45
                if key == pygame.K_d:
                    control.throttle = 0.4
                    control.steer = 0.45
                if key == pygame.K_b:
                    control.brake = 0.7
                    control.throttle = 0.1
            else:
                if key == pygame.K_ESCAPE:
                    return True
                if key == pygame.K_w:
                    control.reverse = False
                    control.throttle = 0.6
                if key == pygame.K_s:
                    control.reverse = True
                    control.throttle = 0.6
                if key == pygame.K_a:
                    control.throttle = 0.4
                    control.steer = -0.45
                if key == pygame.K_d:
                    control.throttle = 0.4
                    control.steer = 0.45
                if key == pygame.K_b:
                    control.brake = 0.7
                    control.throttle = 0.1

        car.apply_control(control)


def control_car_with_wheel(car, joystick, backwards_only=False):
    control = car.get_control()
    control.brake = 0.0
    control.steer = 0.0
    # control.throttle = 0.0

    car.apply_control(control)

    pygame.event.pump()
    move_spectator_to(car.get_transform(), spectator)

    control.steer = max(-1.0, min(1.0, joystick.get_axis(0)))

    throttle = joystick.get_axis(3)
    if throttle >= 0.7:
        throttle = 0.7
    brake = joystick.get_axis(4)
    reverse_button = joystick.get_button(5)

    if backwards_only:
        print("backwards only", throttle)
        control.reverse = True
        control.throttle = throttle
    else:
        if reverse_button:
            control.reverse = True
            control.throttle = throttle
        else:
            control.reverse = False
            control.throttle = throttle


    if brake > 0.1 and brake < 1.0:
        control.brake = brake 

    car.apply_control(control)

    return False

In [47]:
### pedestrian safety mechanism

def send_warning(emergency):
    if emergency:
        mqtt_client.publish(TOPIC, "Pedestrian too close, emergency braking!")
    else:
        mqtt_client.publish(TOPIC, "Pedestrian approaching, slowing down the vehicle.")

def pedestrian_safety_monitoring(vehicle, results, joystick):
    """
    Monitors pedestrian safety using vehicle speed, braking distance, and depth camera data.

    - Stops the vehicle if a pedestrian is too close.
    - Uses depth camera to get accurate pedestrian distances.
    """
    velocity = vehicle.get_velocity()
    vehicle_speed = math.sqrt((velocity.x**2 + velocity.y**2 + velocity.z**2))

    min_safe_distance = 3 # m
    safe_distance = 6 # m

    for confidence, bbox, centroid in results:
        depth = get_depth_at_pixel(int(centroid[0]), int(centroid[1]))
        if depth is None or depth > 100:
            continue

        distance = depth

        global backwards_only_flag
        global braking_flag

        if distance < min_safe_distance and vehicle_speed > 0.0:
            control = vehicle.get_control()
            control.throttle = 0.0
            control.brake = 0.8
            vehicle.apply_control(control)
            send_warning(emergency=True)
            # braking_flag = True
            print("Emergency braking")
            # control_car(vehicle, backwards_only=True)
            control_car_with_wheel(vehicle, joystick, backwards_only=True)
            backwards_only_flag = True
        elif distance < safe_distance and distance > min_safe_distance and vehicle_speed > 0.0:
            control = vehicle.get_control()
            control.throttle = control.throttle / 3
            control.brake = 0.1
            vehicle.apply_control(control)
            send_warning(emergency=False)
            print("Soft emergency braking")
            # control_car(vehicle, backwards_only=False)
            control_car_with_wheel(vehicle, joystick, backwards_only=False)
        elif distance >= min_safe_distance:
            # control_car(vehicle, backwards_only=False)
            control_car_with_wheel(vehicle, joystick, backwards_only=False)
            braking_flag = False
            backwards_only_flag = False

def pedestrian_detection(image, model, vehicle, joystick):
    """
    Detects pedestrians in the image using YOLOv8.
    Only detections with class 'person' (class index 0) and confidence > 0.2 are considered.
    """
    results = model(image)[0]
    detections = []
    if results.boxes is not None and len(results.boxes) > 0:
        boxes = results.boxes.xyxy.cpu().numpy()
        confs = results.boxes.conf.cpu().numpy()
        classes = results.boxes.cls.cpu().numpy()
        for i in range(len(boxes)):
            if int(classes[i]) == 0 and confs[i] > 0.2:
                x1, y1, x2, y2 = boxes[i]
                bbox = (int(x1), int(y1), int(x2), int(y2))
                centroid = ((int(x1) + int(x2)) // 2, (int(y1) + int(y2)) // 2)
                detections.append((confs[i], bbox, centroid))
    pedestrian_safety_monitoring(vehicle, detections, joystick)
    return detections

In [48]:
pygame.joystick.init() # only for joystick control
if pygame.joystick.get_count() == 0:
    print("No joystick detected! Please connect your Logitech steering wheel.")
    exit(1)
joystick = pygame.joystick.Joystick(0) # assuming only one steering wheel is connected
joystick.init()

def main():
    try:
        vehicle = setup_car()
        time.sleep(2)
        control = vehicle.get_control()
        control.brake = 0.0
        control.steer = 0.0
        vehicle.apply_control(control)
        move_spectator_to(vehicle.get_transform(), spectator)

        rgb_camera, depth_camera = setup_camera(vehicle)

        walkers = []
        controllers = []
        for _ in range(50):
            walker, controller = spawn_walker(world)
            if walker and controller:
                walkers.append(walker)
                controllers.append(controller)

        pygame.init()
        pygame.display.set_mode((200,200))

        display = pygame.display.set_mode((VIEW_WIDTH, VIEW_HEIGHT), pygame.HWSURFACE | pygame.DOUBLEBUF)

        model = YOLO("yolov8n.pt")

        while True:
            world.tick()
            control = vehicle.get_control()
            if control.throttle == -1.0:
                control.brake = 0.0
                control.throttle = 0.0
                vehicle.apply_control(control)
            while stored_rgb_image is None or stored_depth_image is None:
                world.tick()

            # raw_image = render(stored_rgb_image)
            # raw_image = cv2.cvtColor(raw_image, cv2.COLOR_BGR2RGB)

            raw_image = render(stored_rgb_image)
            raw_image = raw_image.copy()

            bgr_for_display = cv2.cvtColor(raw_image, cv2.COLOR_RGB2BGR)

            detections = pedestrian_detection(raw_image, model, vehicle, joystick)

            for conf, bbox, centroid in detections:
                cv2.rectangle(bgr_for_display, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0,255,0), 2)

            cv2.imshow("Detection", bgr_for_display)


            pygame.display.flip()
            control_car_with_wheel(vehicle, joystick, backwards_only=backwards_only_flag)
            # control_car(vehicle, backwards_only=backwards_only_flag)
    except KeyboardInterrupt:
        print('\nSimulation interrupted by user')

    finally:
        print('Cleaning up...')
        pygame.quit()
        cv2.destroyAllWindows()
        mqtt_client.disconnect()
        actors = world.get_actors()
        if 'rgb_camera' in locals():
            rgb_camera.destroy()
        if 'depth_camera' in locals():
            depth_camera.destroy()
        actors = world.get_actors()
        for actor in actors:
            if isinstance(actor, (carla.Vehicle, carla.Walker)):
                actor.destroy()
        print('Done.')

if __name__ == '__main__':
    try:
        main()
    except Exception as e:
        traceback.print_exc()
        print(f'Exception occurred: {e}')

Soft emergency braking
Soft emergency braking
Soft emergency braking
Soft emergency braking
Soft emergency braking
Soft emergency braking
Soft emergency braking
Soft emergency braking
Soft emergency braking
Soft emergency braking
Soft emergency braking
Soft emergency braking
Soft emergency braking
Emergency braking
backwards only -1.0
backwards only -1.0
Emergency braking
backwards only -1.0
backwards only -1.0
Emergency braking
backwards only -1.0
backwards only -1.0
Emergency braking
backwards only -1.0
backwards only -1.0
Emergency braking
backwards only -1.0
backwards only -1.0
Emergency braking
backwards only -1.0
backwards only -1.0
Emergency braking
backwards only -1.0
backwards only -1.0
Emergency braking
backwards only -1.0
backwards only -1.0
Emergency braking
backwards only -1.0
backwards only -1.0
Emergency braking
backwards only -1.0
backwards only -1.0
Emergency braking
backwards only -1.0
backwards only -1.0
Emergency braking
backwards only -1.0
backwards only -1.0
Emerg

In [ ]:
actors = world.get_actors()
for actor in actors:
    if isinstance(actor, carla.Vehicle) or isinstance(actor, carla.Walker):
        actor.destroy()